# 데이터 정제 withcolumn과 같이 이용하면 좋은 pyspark.sql.fuction

- spark 도큐먼트 : https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.col.html
1. F.from_json
2. F.udf
3. regexp_replace (or regexp_extract)
4. when, split 절
5. F.to_timestamp and trim

번외)
regexp_replace
.agg(F.countDistinct)

# withcolumn method
 - withcolumn을 이용하여 "기존 컬럼"의 업데이트, 타입변경, 신규컬럼 값 추가
 - withcoumn("new_col", "col")
 - 기존컬럼을 기반으로 신규값 생성 or 업데이트 값 생성시 col("컬럼명")을 이용

# 기본적인 쓰임
 - dataframe.withColumn('추가할 컬럼이름', 연산(col('기존컬럼'))).select(*)

In [ ]:
#df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["age", "name"])
#df.withColumn('age2', col('age') + 2).show()
#+---+-----+----+
#|age| name|age2|
#+---+-----+----+
#|  2|Alice|   4|
#|  5|  Bob|   7|
#+---+-----+----+

# 1.F.from_json
- nested한 컬럼내의 json 을 추출해낼 수 있다.
- parameter = (col , schema, option)
- return 값으로 해당 컬럼의 schema를 따라서 json을 추출해낸다.

In [ ]:
#  기존컬럼을 기반으로 신규값 생성
#  업데이트 값 생성시 col("컬럼명")을 이용 with from_json

from pyspark.sql.fuctions import col
import pyspark.sql.fuctions as F

# 새로운 컬럼 new_col 을 생성하지는 않고 schema에 Struct된 컬럼이 생성된다.
new_df = df.withcolumn('new_col', F.from_json(col('col1', schema = schema)))

# 2. F.udf
 - DataFrame의 특정 컬럼을 사용자가 정의한 함수로 가공할 수 있게 해준다.
 - parameter = (fuction , DataType())

In [ ]:
# UDF 명시
# name을 input 으로 넣으면 "/" 기준 두번째 단어를 선택하겠다.
udf_check_repo_name = F.udf(lambda name : name.split("/")[1], StringType())

# repo_name 컬럼을 생성하는데, UDF를 따라 name 컬럼에'만' 적용해서 생성하겠다)
gitdata = gitdata.withColumn('repo_name', udf_check_repo_name(F.col('name')))


# 3. F.regexp_replace (or regexp_extract)
 - 문자열의 존재 여부 확인 혹은 모든 문자열을 치환할때 """정규표현식"""
 - parameter = (string or dataFrame.columns , "바뀌기 전 대상" , "변환후 대상")
 - return 값은 sub string이 변환된 데이터프레임

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("regexp_replace").getOrCreate()

address = [("RJFK-SLFKW-DFG1T","M BS", "1990-01-13"),
    ("aedw-dg93r-d62g1","W SE", "2000-11-30"),
    ("DFGE-FD23k-DA4G1", "M GJ", "1999-10-08")]
df =spark.createDataFrame(address,["id","demo","birth"])

df.show()

#+----------------+----+----------+
#|              id|demo|     birth|
#+----------------+----+----------+
#|RJFK-SLFKW-DFG1T|M BS|1990-01-13|
#|aedw-dg93r-d62g1|W SE|2000-11-30|
#|DFGE-FD23k-DA4G1|M GJ|1999-10-08|
#+----------------+----+----------+

import pyspark.sql.functions as F
df = df.withColumn("id", F.regexp_replace("id", "-", ""))
df.show()

#+--------------+----+----------+
#|            id|demo|     birth|
#+--------------+----+----------+
#|RJFKSLFKWDFG1T|M BS|1990-01-13|
#|aedwdg93rd62g1|W SE|2000-11-30|
#|DFGEFD23kDA4G1|M GJ|1999-10-08|
#+--------------+----+----------+

# 4. when, split 절



In [ ]:
from pyspark.sql.functions import when, split

df = df.withColumn("sex", \
    when(df.demo.startswith("M"), regexp_replace(df.demo, "M", "Man"))\
    .when(df.demo.startswith("W"), regexp_replace(df.demo, "W", "Woman")))\
    .withColumn("location",\
    when(df.demo.endswith("BS"), regexp_replace(df.demo, "BS", "Busan"))\
    .when(df.demo.endswith("SE"), regexp_replace(df.demo, "SE", "Seoul"))\
    .when(df.demo.endswith("GJ"), regexp_replace(df.demo, "GJ", "Gwangju"))\
    .otherwise(df.demo))

df.show()

#+----------------+----+----------+--------+---------+
#|              id|demo|     birth|     sex| location|
#+----------------+----+----------+--------+---------+
#|RJFK-SLFKW-DFG1T|M BS|1990-01-13|  Man BS|  M Busan|
#|aedw-dg93r-d62g1|W SE|2000-11-30|Woman SE|  W Seoul|
#|DFGE-FD23k-DA4G1|M GJ|1999-11-08|  Man GJ|M Gwangju|
#+----------------+----+----------+--------+---------+

df = df.withColumn("sex", split(df['sex'], ' ').getItem(0))\
        .withColumn("location", split(df['location'], ' ').getItem(1))

df.show()

#+----------------+----+----------+-----+--------+
#|              id|demo|     birth|  sex|location|
#+----------------+----+----------+-----+--------+
#|RJFK-SLFKW-DFG1T|M BS|1990-01-13|  Man|   Busan|
#|aedw-dg93r-d62g1|W SE|2000-11-30|Woman|   Seoul|
#|DFGE-FD23k-DA4G1|M GJ|1999-11-08|  Man| Gwangju|
#+----------------+----+----------+-----+--------+

# 5. F.to_timestamp
 - parameter = (col , format'yyyy-MM-dd HH:mm:ss')
 - data type 이 date time 으로 변환

 # 6. F.trim
 - parameter = col or str target column to work on
 - Returns ==> trimmed values from both sides